# Using Toxic Comment Classification

Here I have used the multi-label Toxic Comment Classification Dataset (by Jigsaw).

I have used Roberta-base to predict 6 different outputs each belonging to one label (toxic, severe_toxic, etc...)

I have not passed the output logits through any sigmoid function. Just used the outputs for these 6 labels and taken a linear average of them.

That's what I have used as the score for comparison.

#### I have used:

* BCELogitsLoss as the loss funtion : It already applies a sigmoid function on the output before calculating the loss
* Early stooping with a patience of 1 : You can modify according to your need
* Used CosineAnnealingLR scheduler : It changes the LR per step following a cosine funtion.


## The final layer of the model looks like this:

 ![NN](https://user-images.githubusercontent.com/74188336/141213710-3a1b7473-8436-4683-841e-64d87789f47e.png)

It has 6 output heads giving outputs to the 6 different labels to determine.

This 6 different heads are attached on top the roberta-base (for now) will implement roberta-large too.

# Aleron's review 👈


### Будь <span style="color:red">Внимательнее</span> Есть недочеты:)

* Используется датасет прошлых лет.
* Несовсем оптимально написана лос функция. 
* Не сохраняется токенайзер
* Некоторые фазый надо вынести в отдельный ноутбук (чистк и подготовка данных)


Model used : RoBERTa-base (RoBERTa-large is way too big and takes a whole lot of time training O.o)

# Инференс модели тут: [ Инференс | Final_blending_all_models | 😎](https://www.kaggle.com/aleron751/final-blending-all-models/edit) 

**Было получено на одном фолде 0.798 на LB (made public)**

In [ ]:
!pip install tez -q
!pip install iterative-stratification -q

In [ ]:
import gc
import os
import tez
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import transformers
from transformers import AdamW, AutoTokenizer, AutoModel
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
import optuna
import time

# Для подготовки данных
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import nltk
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
import re

# Подготовим внешний датасет | Андрей 🔥

In [ ]:
df = pd.read_csv('../input/toxic-comments/train.csv')
df.columns

# Чистка данных | Егор 🔥🔥

In [ ]:
def washing_machine(comments): # Чистка текста от мусора
    corpus=[]
    for i in tqdm(range(len(comments))):
        comment = re.sub('[^a-zA-Z]', ' ', comments[i])
        comment = comment.lower()
        comment = comment.split()
        stemmer = SnowballStemmer('english')
        lemmatizer = WordNetLemmatizer()
        all_stopwords = stopwords.words('english')
        comment = [stemmer.stem(word) for word in comment if not word in set(all_stopwords)]
        comment = [lemmatizer.lemmatize(word) for word in comment]
        comment = ' '.join(comment)
        corpus.append(comment)

    return corpus

df['cleaned_comment_text'] = washing_machine(df['comment_text'].values)

In [ ]:
def create_folds(data, num_splits):
    data.loc[:,'kfold'] = -1
    X = data['cleaned_comment_text']
    y = data[['toxic', 'severe_toxic', 'obscene', 'threat',
           'insult', 'identity_hate']]
    mskf = MultilabelStratifiedKFold(n_splits=num_splits, shuffle=True, random_state=42)
    
    for fold, (trn_, val_) in enumerate(mskf.split(X,y)):
        data.loc[val_,'kfold'] = fold
        
    return data

In [ ]:
df_5 = create_folds(df.copy(), 5)
df_5.to_csv('5folds.csv', index=False)

df_10 = create_folds(df.copy(), 10)
df_10.to_csv('10folds.csv', index=False)

df_5.head()

In [ ]:
df_5['kfold'].value_counts()

# Config

In [ ]:
class Config:
    model_name = 'roberta-base'
    batch_size = 96
    lr = 1e-4
    weight_decay = 0.01
    scheduler = 'CosineAnnealingLR'
    early_stopping_epochs = 1
    epochs = 15 # 20
    max_length = 128
    #max_length = 196
    num_folds = 2

# Dataset

In [ ]:
class ToxicDataset:
    def __init__(self, data, tokenizer,  max_len=196):
        self.comments = data['comment_text'].values
        self.tokenizer = tokenizer
        self.targets = data[[
            'toxic', 'severe_toxic', 'obscene',
            'threat','insult', 'identity_hate']].values
        self.max_len = max_len
        
    def __len__(self):
        return len(self.comments)
    
    def __getitem__(self, idx):
        
        tokenized = self.tokenizer.encode_plus(
            self.comments[idx],
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length'
        )
        
        input_ids = tokenized['input_ids']
        attention_mask = tokenized['attention_mask']
        
        toxic, severe_toxic, obscene, threat, insult, identity_hate = self.targets[idx]

        return {
            'input_ids' : torch.tensor(input_ids, dtype=torch.long),
            'attention_mask' : torch.tensor(attention_mask, dtype=torch.long),
            'toxic' : torch.tensor(toxic, dtype=torch.float),
            'severe_toxic' : torch.tensor(severe_toxic, dtype=torch.float),
            'obscene' : torch.tensor(obscene, dtype=torch.float),
            'threat' : torch.tensor(threat, dtype=torch.float),
            'insult' : torch.tensor(insult, dtype=torch.float),
            'identity_hate' : torch.tensor(identity_hate, dtype=torch.float)
        }

# The Model

In [ ]:
class ToxicModel(nn.Module):
    def __init__(self, args, model_name):
        super(ToxicModel, self).__init__()
        self.args = args
        self.model = AutoModel.from_pretrained(self.args.model_name)
        self.dropout = nn.Dropout(p=0.2)
        self.toxic = nn.Linear(768, 1)
        self.stoxic = nn.Linear(768, 1)
        self.obs = nn.Linear(768, 1)
        self.threat = nn.Linear(768, 1)
        self.insult = nn.Linear(768, 1)
        self.id_hate = nn.Linear(768, 1)
    
        
    def forward(self, input_ids, attention_mask):
        
        out = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=False
        )
        
        out = self.dropout(out[1])
        
        toxic = self.toxic(out)
        stoxic = self.stoxic(out)
        obs = self.obs(out)
        threat = self.threat(out)
        insult = self.insult(out)
        id_hate = self.id_hate(out)

        return [toxic, stoxic, obs, threat, insult, id_hate]
        

# Loss Function

In [ ]:
def loss_fn(outputs, targets):
    o1, o2, o3, o4, o5, o6 = outputs
    t1, t2, t3, t4, t5, t6 = targets
    l1 = nn.BCEWithLogitsLoss()(o1, t1.view(-1,1))
    l2 = nn.BCEWithLogitsLoss()(o2, t2.view(-1,1))
    l3 = nn.BCEWithLogitsLoss()(o3, t3.view(-1,1))
    l4 = nn.BCEWithLogitsLoss()(o4, t4.view(-1,1))
    l5 = nn.BCEWithLogitsLoss()(o5, t5.view(-1,1))
    l6 = nn.BCEWithLogitsLoss()(o6, t6.view(-1,1))
    
    total_loss = (l1+l2+l3+l4+l5+l6)/6
    
    return total_loss

In [ ]:
def metrics(outputs, targets):
    auc_scores=[]
    for o, t in zip(outputs, targets):
        o = o.cpu().detach().numpy()
        t = t.cpu().detach().numpy()
        auc = roc_auc_score(o, t)
        auc_scores(auc)

    return np.mean(auc_scores)

# Training epoch

In [ ]:
def train_epoch(args, dataloader, model, optimizer, scheduler, epoch):
    model.train()
    epoch_loss = 0.0
    running_loss = 0.0
    dataset_size=0
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        optimizer.zero_grad()
        
        input_ids = data['input_ids'].cuda()
        attention_mask = data['attention_mask'].cuda()
        toxic = data['toxic'].cuda()
        severe_toxic = data['severe_toxic'].cuda()
        obscene = data['obscene'].cuda()
        threat = data['threat'].cuda()
        insult = data['insult'].cuda()
        identity_hate = data['identity_hate'].cuda()
        
        batch_size = args.batch_size
        
        targets = (toxic, severe_toxic, obscene, threat, insult, identity_hate)
        outputs = model(input_ids, attention_mask)
        
        
        loss = loss_fn(outputs, targets)

        
        loss.backward()
        optimizer.step()
        if scheduler is not None:
            scheduler.step()
        
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        epoch_loss = running_loss / dataset_size
        
        bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])
    gc.collect()
    return epoch_loss

# Validation Epoch

In [ ]:
def validation(args, dataloader, model):
    model.eval()
    epoch_loss = 0.0
    running_loss = 0.0
    dataset_size=0
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    with torch.no_grad():
        for step, data in bar:
            batch_size = args.batch_size

            input_ids = data['input_ids'].cuda()
            attention_mask = data['attention_mask'].cuda()
            toxic = data['toxic'].cuda()
            severe_toxic = data['severe_toxic'].cuda()
            obscene = data['obscene'].cuda()
            threat = data['threat'].cuda()
            insult = data['insult'].cuda()
            identity_hate = data['identity_hate'].cuda()

            targets = (toxic, severe_toxic, obscene, threat, insult, identity_hate)

            outputs = model(input_ids, attention_mask)

            loss = loss_fn(outputs, targets)

            running_loss += (loss.item() * batch_size)
            dataset_size += batch_size

            epoch_loss = running_loss / dataset_size

            bar.set_postfix(Valid_Loss=epoch_loss,
                            Stage='Validation') 
    return epoch_loss

# Optimizer

In [ ]:
def get_optimizer(args, params):
    opt = AdamW(params, lr=args.lr, weight_decay=args.weight_decay)
    return opt

# Scheduler

In [ ]:
def get_scheduler(args, optimizer):
    if args.scheduler == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=500, 
                                                   eta_min=1e-6)
    else:
        schduler = None
    return scheduler

# Training and validation Loop

In [ ]:
def run(data, fold, args=None, save_model=False):
    print('-'*50)
    print(f'Fold : {fold}')
    print('-'*50)
    
    if args is None:
        args = Config()
        
    start = time.time()
    model = ToxicModel(args, args.model_name)
    model = model.cuda()
    tokenizer = AutoTokenizer.from_pretrained(args.model_name)
    
    optimizer = get_optimizer(args, model.parameters())
    scheduler = get_scheduler(args, optimizer)
    
    train = data[data['kfold']!=fold]
    valid = data[data['kfold']==fold]
    
    train_dataset = ToxicDataset(train, tokenizer, args.max_length)
    valid_dataset = ToxicDataset(valid, tokenizer, args.max_length)
    
    train_loader = DataLoader(train_dataset, batch_size=args.batch_size)
    valid_loader = DataLoader(valid_dataset, batch_size=2*args.batch_size)
    
    best_val_loss = np.inf
    patience_counter = 0

    for epoch in range(args.epochs):
        
        train_loss = train_epoch(args, train_loader, model, optimizer, scheduler, epoch)
        valid_loss = validation(args, valid_loader, model)
        
        if valid_loss <= best_val_loss:
            print(f"Validation Loss Improved ({best_val_loss} ---> {valid_loss})")
            best_val_loss = valid_loss
            
            if save_model:
                PATH = f"model_fold_{fold}.bin"
                torch.save(model.state_dict(), PATH)
                print(f"----------Model Saved----------")
        
        else:
            patience_counter += 1
            print(f'Early stopping counter {patience_counter} of {args.early_stopping_epochs}')
            if patience_counter == args.early_stopping_epochs:
                print('*************** Early Stopping ***************')
                break
    
    end = time.time()
    time_elapsed = end-start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best Loss: {:.4f}".format(best_val_loss))
    
    del model, train_loader, valid_loader
    gc.collect()
    return best_val_loss

# Read the data

I have used 5 folds that I have cleaned using this notebook: [Multi-Label Stratified K-fold | Toxic Comments](https://www.kaggle.com/kishalmandal/multi-label-stratified-k-fold-toxic-comments)

The data is the same data used in the toxicity classification challenge by Jigsaw

In [ ]:
df = pd.read_csv('5folds.csv')
df.head()

In [ ]:
df = df.dropna()

In [ ]:
df.isnull().sum()

# Optuna

In [ ]:
# df_trial = df[:100]

In [ ]:
# def objective(trial):
#     args = Config()
#     args.epochs=1
#     args.lr = trial.suggest_uniform('lr',1e-6, 1e-3)
#     all_losses = []
#     for fold in range(5):
#         temp_loss = run(df_trial, fold, args=args)
#         all_losses.append(temp_loss)
    
#     return np.mean(all_losses)

In [ ]:
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=10)

# print('Best Trial:')
# trial_ = study.best_trial
# print(trial_.values)

In [ ]:
# trial_.params['lr']

# Run training

In [ ]:
args=Config()
args.lr = 0.0005149849355804644

# Пробегаемся по всем фолдам
# run(df, fold=0, save_model=True, args=args)

for fold in tqdm(range(5)):
    run(df, fold=fold, save_model=True, args=args)


# Готовим submission | Inference

In [ ]:
# com1 = washing_machine(df['less_toxic'].values)
# com2 = washing_machine(df['more_toxic'].values)

In [ ]:
def get_predictions(args, dataloader, model):
    model.eval()
    all_outputs=[]
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    with torch.no_grad():
        for step, data in bar:
            batch_size = args.batch_size

            input_ids = data['input_ids'].cuda()
            attention_mask = data['attention_mask'].cuda()
            outputs = model(input_ids, attention_mask)
            outputs = outputs.cpu().detach().numpy()
            outputs = [sum(output) for output in outputs]
            all_outputs.append(outputs)

            bar.set_postfix(Stage='Inference') 
    return np.hstack(all_outputs)

In [ ]:
def inference(data):
    args=Config()
    tokenizer = AutoTokenizer.from_pretrained(args.model_name)
    base_path='./'
    
    dataset = ToxicDataset(data, tokenizer)
    dataloader = DataLoader(dataset, batch_size=16*args.batch_size)
    
    final_preds = []
    
    num_folds = args.num_folds
    
    for fold in range(num_folds):
        model = ToxicModel(args)
        model = model.cuda()
        path = base_path + f'model_fold_{fold}.bin'
        model.load_state_dict(torch.load(path))
        
        print(f"Getting predictions for model {fold+1}")
        preds = get_predictions(args, dataloader, model)
        final_preds.append(np.vstack(preds))
        del model
        gc.collect()
    return np.hstack(sum(final_preds)/num_folds)

In [ ]:
# pred1 = inference(com1)
# pred2 = inference(com2)

In [ ]:
df = pd.read_csv('../input/jigsaw-toxic-severity-rating/comments_to_score.csv')

In [ ]:
comments = washing_machine(df['text'].values)

In [ ]:
class Config:
    model_name = 'roberta-base'
    batch_size = 96
    lr = 1e-4
    weight_decay = 0.01
    scheduler = 'CosineAnnealingLR'
    early_stopping_epochs = 1
    epochs = 15 # было 20
    max_length = 196 # = 128
    num_folds = 2
    
class ToxicModel(nn.Module):
    def __init__(self, args):
        super(ToxicModel, self).__init__()
        self.args = args
        self.model = AutoModel.from_pretrained(self.args.model_name)
        self.dropout = nn.Dropout(p=0.2)
        self.toxic = nn.Linear(768, 1)
        self.stoxic = nn.Linear(768, 1)
        self.obs = nn.Linear(768, 1)
        self.threat = nn.Linear(768, 1)
        self.insult = nn.Linear(768, 1)
        self.id_hate = nn.Linear(768, 1)
    
        
    def forward(self, input_ids, attention_mask):
        
        out = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=False
        )
        
        out = self.dropout(out[1])
        
        toxic = self.toxic(out)
        stoxic = self.stoxic(out)
        obs = self.obs(out)
        threat = self.threat(out)
        insult = self.insult(out)
        id_hate = self.id_hate(out)

        return torch.cat([toxic, stoxic, obs, threat, insult, id_hate], dim=-1)
            
    
class ToxicDataset:
    def __init__(self, comments, tokenizer, max_len=196):
        self.comments = comments
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.comments)
    
    def __getitem__(self, idx):
        
        tokenized = self.tokenizer.encode_plus(
            self.comments[idx],
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length'
        )
        
        input_ids = tokenized['input_ids']
        attention_mask = tokenized['attention_mask']
        

        return {
            'input_ids' : torch.tensor(input_ids, dtype=torch.long),
            'attention_mask' : torch.tensor(attention_mask, dtype=torch.long)
        }

In [ ]:
pred = inference(comments)

In [ ]:
df['score'] = pred

In [ ]:
df[['comment_id', 'score']].to_csv('submission.csv', index=False)